In [1]:
# Cell 1: Imports and Configuration
# Description: This cell imports all necessary libraries and sets up the global configuration variables for the application.

import json
import os
import random
import time
import getpass
import hashlib
import sys
from IPython.display import clear_output

# --- Configuration ---
DATA_DIR = "app_data"
USERS_FILE = os.path.join(DATA_DIR, "users.json")
CONTENT_FILE = os.path.join(DATA_DIR, "content.json")
LEADERBOARD_FILE = os.path.join(DATA_DIR, "leaderboard.json")
STUDY_SESSION_MINUTES = 10
COINS_PER_SESSION = 1
GAME_ACCESS_COST = 5  # coins
GAME_TIME_MINUTES = 10
AVATAR_UNLOCK_COST = 10

# Note: getpass might not work in all Jupyter environments. 
# If it fails, passwords will be visible during input.


In [2]:
# Cell 2: Helper Functions
# Description: These are utility functions for file management, data handling, and screen clearing.

def clear_screen_notebook():
    """Clears the cell output in a Jupyter Notebook."""
    clear_output(wait=True)

def setup_data_files():
    """Creates necessary data files and directories if they don't exist."""
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    if not os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'w') as f:
            json.dump({}, f)

    if not os.path.exists(LEADERBOARD_FILE):
        with open(LEADERBOARD_FILE, 'w') as f:
            json.dump({}, f)

    if not os.path.exists(CONTENT_FILE):
        # In a real app, this content would be vast.
        # Here's a small, structured sample.
        content = {
            "AQA": {
                "Maths": {
                    "Algebra": ["Solving linear equations.", "Understanding quadratic expressions.", "Factorising quadratics."],
                    "Geometry": ["Calculating area of a circle.", "Pythagoras' theorem.", "Trigonometry basics (SOH CAH TOA)."]
                },
                "Physics": {
                    "Energy": ["Types of energy stores.", "Energy transfers.", "Calculating kinetic energy."],
                    "Forces": ["Newton's First Law.", "Calculating work done.", "Hooke's Law."]
                }
            },
            "Edexcel": {
                 "Maths": {
                    "Algebra": ["Simultaneous equations.", "The quadratic formula.", "Inequalities."],
                    "Geometry": ["Circle theorems.", "Vectors.", "3D shapes."]
                },
                "Chemistry": {
                    "Atomic Structure": ["Protons, neutrons, electrons.", "Isotopes.", "Electron shell configuration."],
                    "The Periodic Table": ["Groups and periods.", "Properties of alkali metals.", "Properties of halogens."]
                }
            }
            # ... More boards and subjects would be added here
        }
        with open(CONTENT_FILE, 'w') as f:
            json.dump(content, f, indent=4)


def load_data(file_path):
    """Loads JSON data from a file."""
    with open(file_path, 'r') as f:
        return json.load(f)

def save_data(data, file_path):
    """Saves data to a JSON file."""
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def hash_password(password):
    """Hashes a password for secure storage."""
    return hashlib.sha256(password.encode()).hexdigest()

print("Helper functions defined.")


Helper functions defined.


In [3]:
# Cell 3: Game Implementation (Snake)
# Description: This cell contains the complete code for the Snake game, using the pygame library.
# Note: Pygame runs in a separate window, not within the notebook's output cell.

def run_snake_game():
    """A simple implementation of the Snake game."""
    try:
        import pygame
    except ImportError:
        print("\nPygame is required to play games. Please install it using: pip install pygame")
        input("Press Enter to return to the main menu...")
        return

    pygame.init()

    # Screen dimensions
    width, height = 600, 400
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Snake')

    # Colors
    black = (0, 0, 0)
    white = (255, 255, 255)
    red = (213, 50, 80)
    green = (0, 255, 0)

    # Snake and food properties
    snake_block = 10
    snake_speed = 15

    clock = pygame.time.Clock()
    font_style = pygame.font.SysFont(None, 30)

    def message(msg, color):
        mesg = font_style.render(msg, True, color)
        # Center the message
        text_rect = mesg.get_rect(center=(width / 2, height / 2))
        screen.blit(mesg, text_rect)

    def gameLoop():
        game_over = False
        game_close = False

        x1, y1 = width / 2, height / 2
        x1_change, y1_change = 0, 0

        snake_List = []
        Length_of_snake = 1
        score = 0

        foodx = round(random.randrange(0, width - snake_block) / 10.0) * 10.0
        foody = round(random.randrange(0, height - snake_block) / 10.0) * 10.0

        while not game_over:
            while game_close:
                screen.fill(black)
                message(f"You Lost! Score: {score}. Press Q-Quit or C-Play Again", red)
                pygame.display.update()

                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_q:
                            pygame.quit()
                            return score
                        if event.key == pygame.K_c:
                            # This call might be problematic in some setups, a full restart is safer
                            return gameLoop() 

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_over = True
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_LEFT:
                        x1_change = -snake_block
                        y1_change = 0
                    elif event.key == pygame.K_RIGHT:
                        x1_change = snake_block
                        y1_change = 0
                    elif event.key == pygame.K_UP:
                        y1_change = -snake_block
                        x1_change = 0
                    elif event.key == pygame.K_DOWN:
                        y1_change = snake_block
                        x1_change = 0

            if x1 >= width or x1 < 0 or y1 >= height or y1 < 0:
                game_close = True
            x1 += x1_change
            y1 += y1_change
            screen.fill(black)
            pygame.draw.rect(screen, red, [foodx, foody, snake_block, snake_block])
            snake_Head = [x1, y1]
            snake_List.append(snake_Head)
            if len(snake_List) > Length_of_snake:
                del snake_List[0]

            for x in snake_List[:-1]:
                if x == snake_Head:
                    game_close = True

            for segment in snake_List:
                pygame.draw.rect(screen, green, [segment[0], segment[1], snake_block, snake_block])
            
            score_font = pygame.font.SysFont(None, 35)
            value = score_font.render("Your Score: " + str(score), True, white)
            screen.blit(value, [0, 0])

            pygame.display.update()

            if x1 == foodx and y1 == foody:
                foodx = round(random.randrange(0, width - snake_block) / 10.0) * 10.0
                foody = round(random.randrange(0, height - snake_block) / 10.0) * 10.0
                Length_of_snake += 1
                score += 1

            clock.tick(snake_speed)

        pygame.quit()
        return score

    final_score = gameLoop()
    return final_score

print("Snake game function defined.")


Snake game function defined.


In [4]:
# Cell 4: Main Application Class
# Description: This class encapsulates all the core logic of the study application,
# such as user registration, login, study sessions, and accessing the shop and games.

class StudyApp:
    def __init__(self):
        self.current_user = None
        self.users_data = load_data(USERS_FILE)
        self.content_data = load_data(CONTENT_FILE)
        self.leaderboard = load_data(LEADERBOARD_FILE)

    def save_all_data(self):
        """Saves all potentially modified data to files."""
        save_data(self.users_data, USERS_FILE)
        save_data(self.leaderboard, LEADERBOARD_FILE)

    def login(self):
        clear_screen_notebook()
        print("--- Login ---")
        username = input("Enter username: ")
        password = getpass.getpass("Enter password: ")
        hashed_password = hash_password(password)

        if username in self.users_data and self.users_data[username]['password'] == hashed_password:
            self.current_user = username
            print(f"\nWelcome back, {self.current_user}!")
            time.sleep(1.5)
            return True
        else:
            print("\nInvalid username or password.")
            time.sleep(1.5)
            return False

    def register(self):
        clear_screen_notebook()
        print("--- Register ---")
        username = input("Choose a username: ")
        if username in self.users_data:
            print("\nUsername already exists.")
            time.sleep(1.5)
            return

        password = getpass.getpass("Choose a password: ")
        
        self.users_data[username] = {
            "password": hash_password(password),
            "coins": 0,
            "unlocked_avatars": ["Avatar 1", "Avatar 2", "Avatar 3"],
            "unlocked_games": ["Snake"],
            "current_avatar": "Avatar 1"
        }
        self.save_all_data()
        print(f"\nUser '{username}' registered successfully!")
        time.sleep(1.5)

    def start_study_session(self):
        clear_screen_notebook()
        print("--- Study Session ---")
        
        boards = list(self.content_data.keys())
        print("Select an Examination Board:")
        for i, board in enumerate(boards): print(f"{i + 1}. {board}")
        
        try:
            board_choice = int(input("> ")) - 1
            selected_board = boards[board_choice]
        except (ValueError, IndexError):
            print("Invalid choice."); time.sleep(1); return

        subjects = list(self.content_data[selected_board].keys())
        print("\nSelect a Subject:")
        for i, subject in enumerate(subjects): print(f"{i + 1}. {subject}")

        try:
            subject_choice = int(input("> ")) - 1
            selected_subject = subjects[subject_choice]
        except (ValueError, IndexError):
            print("Invalid choice."); time.sleep(1); return

        topics = self.content_data[selected_board][selected_subject]
        topic_name, content_chunks = random.choice(list(topics.items()))
        study_content = random.choice(content_chunks)

        clear_screen_notebook()
        print(f"Board: {selected_board} | Subject: {selected_subject} | Topic: {topic_name}")
        print("-" * 50)
        print(f"Your 10-minute study chunk is:\n\n  - {study_content}\n")
        print("-" * 50)
        print(f"The timer for {STUDY_SESSION_MINUTES} minutes starts NOW.")
        
        try:
            for i in range(STUDY_SESSION_MINUTES * 60, 0, -1):
                mins, secs = divmod(i, 60)
                timer_display = f'{mins:02d}:{secs:02d}'
                print(f"Time remaining: {timer_display}", end='\r'); sys.stdout.flush(); time.sleep(1)
        except KeyboardInterrupt:
            print("\n\nStudy session interrupted."); input("Press Enter..."); return
        
        self.users_data[self.current_user]['coins'] += COINS_PER_SESSION
        self.save_all_data()
        print(f"\n\nTime's up! You've earned {COINS_PER_SESSION} gold coin!")
        input("Press Enter to return to the main menu...")

    def visit_shop(self):
        while True:
            clear_screen_notebook()
            user_coins = self.users_data[self.current_user]['coins']
            print("--- Shop ---")
            print(f"Your coins: {user_coins} 🪙")
            print("\n1. Unlock Avatars\n2. Unlock Games (Coming soon!)\n3. Back to main menu")
            choice = input("> ")
            if choice == '1': self.unlock_avatars()
            elif choice == '2': print("\nMore games are under development!"); time.sleep(2)
            elif choice == '3': break

    def unlock_avatars(self):
        clear_screen_notebook()
        user_coins = self.users_data[self.current_user]['coins']
        unlocked = self.users_data[self.current_user]['unlocked_avatars']
        all_avatars = [f"Avatar {i}" for i in range(1, 11)]
        
        print(f"--- Unlock Avatars ---\nYour coins: {user_coins} 🪙\nCost: {AVATAR_UNLOCK_COST} 🪙 per avatar.\n")
        for i, avatar in enumerate(all_avatars):
            status = " (Unlocked)" if avatar in unlocked else f" ({AVATAR_UNLOCK_COST} 🪙)"
            print(f"{i + 1}. {avatar}{status}")
        print("\n0. Back to Shop")

        try:
            choice = int(input("\nEnter number of avatar to unlock: "))
            if choice == 0: return
            target_avatar_name = all_avatars[choice - 1]

            if target_avatar_name in unlocked: print("\nYou already own this avatar."); time.sleep(1.5)
            elif user_coins >= AVATAR_UNLOCK_COST:
                self.users_data[self.current_user]['coins'] -= AVATAR_UNLOCK_COST
                self.users_data[self.current_user]['unlocked_avatars'].append(target_avatar_name)
                self.save_all_data()
                print(f"\nCongratulations! You've unlocked {target_avatar_name}!"); time.sleep(2)
            else: print("\nNot enough coins!"); time.sleep(1.5)
        except (ValueError, IndexError): print("Invalid choice."); time.sleep(1)

    def view_profile(self):
        clear_screen_notebook()
        user_data = self.users_data[self.current_user]
        print(f"--- Your Profile ---\nUsername: {self.current_user}\nCurrent Avatar: {user_data['current_avatar']}\nGold Coins: {user_data['coins']} 🪙")
        print("\n--- Unlocked Items ---\nAvatars: " + ", ".join(user_data['unlocked_avatars']))
        print("Games: " + ", ".join(user_data['unlocked_games']))
        print("\n1. Change Avatar\n2. Back to Main Menu")
        if input("> ") == '1': self.change_avatar()

    def change_avatar(self):
        clear_screen_notebook()
        unlocked_avatars = self.users_data[self.current_user]['unlocked_avatars']
        print("--- Change Avatar ---")
        for i, avatar in enumerate(unlocked_avatars): print(f"{i + 1}. {avatar}")
        
        try:
            choice = int(input("\nSelect your new avatar: ")) - 1
            if 0 <= choice < len(unlocked_avatars):
                new_avatar = unlocked_avatars[choice]
                self.users_data[self.current_user]['current_avatar'] = new_avatar
                self.save_all_data()
                print(f"\nAvatar changed to {new_avatar}!")
            else: print("Invalid choice.")
        except ValueError: print("Invalid input.")
        time.sleep(1.5)

    def access_games(self):
        clear_screen_notebook()
        user_data = self.users_data[self.current_user]
        print(f"--- Game Zone ---\nYour coins: {user_data['coins']} 🪙")
        print(f"Cost to play: {GAME_ACCESS_COST} coins for {GAME_TIME_MINUTES} minutes of access.")

        if user_data['coins'] < GAME_ACCESS_COST:
            print("\nYou don't have enough coins to play. Time to study!"); time.sleep(2); return

        if input(f"\nSpend {GAME_ACCESS_COST} coins to play? (y/n): ").lower() != 'y': return
            
        self.users_data[self.current_user]['coins'] -= GAME_ACCESS_COST
        self.save_all_data()
        print(f"\nCoins spent. You have {GAME_TIME_MINUTES} minutes of game time. Enjoy!"); time.sleep(2)

        unlocked_games = user_data['unlocked_games']
        print("\nSelect a game to play:")
        for i, game_name in enumerate(unlocked_games): print(f"{i+1}. {game_name}")
        
        try:
            game_choice = int(input("> ")) - 1
            if 0 <= game_choice < len(unlocked_games):
                selected_game = unlocked_games[game_choice].lower()
                if selected_game == "snake":
                    # Game runs in a new window
                    score = run_snake_game() 
                    if score is not None: self.update_leaderboard("Snake", score)
                else: print("This game is not implemented yet."); time.sleep(2)
            else: print("Invalid choice."); time.sleep(1)
        except ValueError: print("Invalid input."); time.sleep(1)

    def update_leaderboard(self, game_name, score):
        if game_name not in self.leaderboard: self.leaderboard[game_name] = []
        self.leaderboard[game_name].append({"username": self.current_user, "score": score})
        self.leaderboard[game_name] = sorted(self.leaderboard[game_name], key=lambda x: x['score'], reverse=True)[:10]
        self.save_all_data()
        print(f"\nScore of {score} submitted to the {game_name} leaderboard!"); time.sleep(2)

    def view_leaderboard(self):
        clear_screen_notebook()
        print("--- Leaderboards ---")
        if not self.leaderboard:
            print("No scores recorded yet. Go play some games!"); time.sleep(2); return

        for game_name, scores in self.leaderboard.items():
            print(f"\n--- {game_name} Top 10 ---")
            if not scores: print("No scores yet for this game."); continue
            for i, entry in enumerate(scores): print(f"{i+1:2}. {entry['username']:<15} - {entry['score']}")
        input("\nPress Enter to return to the main menu...")

    def main_menu(self):
        while True:
            clear_screen_notebook()
            user_data = self.users_data[self.current_user]
            print(f"--- Main Menu ---\nUser: {self.current_user} | Avatar: {user_data['current_avatar']} | Coins: {user_data['coins']} 🪙")
            print("\n1. Start Study Session\n2. Access Game Zone\n3. View Profile & Avatars\n4. Visit Shop\n5. View Leaderboards\n6. Logout")
            choice = input("> ")
            if choice == '1': self.start_study_session()
            elif choice == '2': self.access_games()
            elif choice == '3': self.view_profile()
            elif choice == '4': self.visit_shop()
            elif choice == '5': self.view_leaderboard()
            elif choice == '6': self.current_user = None; print("Logged out."); time.sleep(1); break

    def run(self):
        while True:
            clear_screen_notebook()
            print("--- Welcome to the GCSE Study App ---\n\n1. Login\n2. Register\n3. Exit")
            choice = input("> ")
            if choice == '1':
                if self.login(): self.main_menu()
            elif choice == '2': self.register()
            elif choice == '3': print("Goodbye!"); break

print("StudyApp class defined.")


StudyApp class defined.


In [5]:

# Cell 5: Execution
# Description: This is the final cell. Running it will set up the necessary data files
# and start the application loop. The program will run in the output of this cell.

# Ensure data files are ready
setup_data_files()

# Create an instance of the app
app = StudyApp()

# Run the application
# To stop the app, choose the "Exit" option from the main menu.
app.run()

--- Welcome to the GCSE Study App ---

1. Login
2. Register
3. Exit
Goodbye!
